## Settings

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import helpers.settings as sts
sts.print_settings(sts)

DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model_0.0.1.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7fd5c6aa5c10>



## Imports

In [4]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import numpy as np
from category_encoders.woe import WOEEncoder
import warnings
from datetime import datetime
import pickle
from helpers.processing import ModifiedColumnTransformer

## Load train dataset

In [5]:
df_train = pd.read_parquet(f"data/{sts.DATASET_TRAIN_FILENAME}")

In [6]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

## Features

In [7]:
X_train.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,F,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21645,365243,1,0,0,0,None,2.0
1,F,N,N,0,157500.0,Commercial associate,Higher education,Civil marriage,House / apartment,-10193,-105,1,1,1,0,Accountants,2.0
2,F,Y,Y,0,675000.0,Pensioner,Higher education,Married,House / apartment,-21721,365243,1,0,0,0,None,2.0
3,F,Y,Y,2,112500.0,Working,Secondary / secondary special,Married,House / apartment,-9994,-644,1,1,0,0,Sales staff,4.0
4,F,N,Y,0,112500.0,Pensioner,Lower secondary,Married,House / apartment,-23754,365243,1,0,0,0,None,2.0


In [8]:
excluded_features = []

In [9]:
special_features = []

In [10]:
numeric_features = [
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "CNT_FAM_MEMBERS",
]

In [11]:
categorical_features = [
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "FLAG_MOBIL",    
    "FLAG_WORK_PHONE",
    "FLAG_PHONE",
    "FLAG_EMAIL",
    "OCCUPATION_TYPE",
]

In [12]:
for numeric_feature in numeric_features:
    print(f"{numeric_feature}: {numeric_feature in X_train.select_dtypes(exclude=['object']).columns}")

CNT_CHILDREN: True
AMT_INCOME_TOTAL: True
DAYS_BIRTH: True
DAYS_EMPLOYED: True
CNT_FAM_MEMBERS: True


In [13]:
for categorical_feature in categorical_features:
    print(f"{categorical_feature}:{categorical_feature in X_train.select_dtypes(include=['object']).columns}")

CODE_GENDER:True
FLAG_OWN_CAR:True
FLAG_OWN_REALTY:True
NAME_INCOME_TYPE:True
NAME_EDUCATION_TYPE:True
NAME_FAMILY_STATUS:True
NAME_HOUSING_TYPE:True
FLAG_MOBIL:False
FLAG_WORK_PHONE:False
FLAG_PHONE:False
FLAG_EMAIL:False
OCCUPATION_TYPE:True


## Pipeline

In [14]:
categorical_transformer = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="constant",missing_values=np.nan, fill_value=np.nan)),
        ("encoder", WOEEncoder()),
    ]
)

numeric_transformer = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="constant", missing_values=np.nan, fill_value=np.nan)),
        ("scaler", RobustScaler()),
    ]
)


preprocessor = ModifiedColumnTransformer(
    transformers=[
        ("categorical", categorical_transformer, categorical_features),
        ("numeric", numeric_transformer, numeric_features),
    ],
    numeric_features=numeric_features,
    categorical_features=categorical_features,
)

<ipython-input-14-cb27ffeb604a>:16: UserWarning: 2021-08-30 20:23:38.540433 INFO: Hard mode for the ModifiedColumnTransformer set to True: The initial features are going to be enforced during transformation and fit steps
  preprocessor = ModifiedColumnTransformer(


In [15]:
pd.DataFrame(preprocessor.fit_transform(X_train,y_train), columns=preprocessor.final_features)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,0.043373,-0.017715,0.248458,0.363272,0.026367,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,-0.217391,-0.871195,133.819774,0.0
1,0.043373,-0.017715,-0.841560,-0.073486,-0.215596,-0.439822,-0.041703,-0.002433,-0.528348,0.076444,-0.057213,0.606423,0.0,0.000000,0.773262,0.529734,0.0
2,0.043373,0.028455,0.248458,0.363272,-0.215596,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,5.000000,-0.882108,133.819774,0.0
3,0.043373,0.028455,0.248458,-0.027511,0.026367,-0.013086,-0.041703,-0.002433,-0.528348,-0.033699,-0.057213,-0.193711,2.0,-0.434783,0.801838,0.333090,2.0
4,0.043373,-0.017715,0.248458,0.363272,-0.341495,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,-0.434783,-1.174038,133.819774,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30983,0.043373,-0.017715,0.248458,0.363272,0.026367,0.174109,-0.041703,-0.002433,0.112936,0.076444,0.447095,0.000000,0.0,-0.434783,-1.123205,133.819774,-1.0
30984,0.043373,-0.017715,0.248458,-0.073486,0.026367,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,-0.431977,1.0,-0.217391,0.006605,-2.443999,1.0
30985,0.043373,-0.017715,0.248458,-0.027511,0.026367,-0.013086,-0.041703,-0.002433,-0.528348,0.076444,-0.057213,-0.305770,0.0,-0.782609,-0.316485,-0.632616,0.0
30986,0.043373,-0.017715,0.248458,0.363272,0.026367,-0.439822,-0.041703,-0.002433,0.112936,0.076444,-0.057213,0.000000,0.0,0.000000,-0.869759,133.819774,0.0


## Dump preprocessor

In [16]:
with open(f"data/{sts.PREPROCESSOR_FILENAME}","wb") as file:
    pickle.dump(preprocessor, file)